LOADING LIBRAIES AND FUNCTIONS 

In [2]:
import cv2 
import numpy as np
import pandas as pd
import os 
from imutils import paths

from keras.applications import MobileNetV2
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam 

from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from keras_preprocessing.image import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

LOADING IN THE DATASET 

In [3]:
dataset = "dataset"

PATHS ARE LISTED  IN A LIST imagePaths

In [4]:
imagePaths=list(paths.list_images(dataset))

PRINT IMAGE PATHS

In [5]:
imagePaths

['dataset\\without_mask\\without_mask_100.jpg',
 'dataset\\without_mask\\without_mask_1000.jpg',
 'dataset\\without_mask\\without_mask_1001.jpg',
 'dataset\\without_mask\\without_mask_1002.jpg',
 'dataset\\without_mask\\without_mask_1004.jpg',
 'dataset\\without_mask\\without_mask_1005.jpg',
 'dataset\\without_mask\\without_mask_1009.jpg',
 'dataset\\without_mask\\without_mask_101.jpg',
 'dataset\\without_mask\\without_mask_1011.jpg',
 'dataset\\without_mask\\without_mask_1013.jpg',
 'dataset\\without_mask\\without_mask_1014.jpg',
 'dataset\\without_mask\\without_mask_1015.jpg',
 'dataset\\without_mask\\without_mask_1017.jpg',
 'dataset\\without_mask\\without_mask_1019.jpg',
 'dataset\\without_mask\\without_mask_1020.jpg',
 'dataset\\without_mask\\without_mask_1021.jpg',
 'dataset\\without_mask\\without_mask_1022.jpg',
 'dataset\\without_mask\\without_mask_1024.jpg',
 'dataset\\without_mask\\without_mask_1027.jpg',
 'dataset\\without_mask\\without_mask_1028.jpg',
 'dataset\\without_mas

LABELS ARE GIVEN TO THE IMAGE AND IMAGES ARE THEN -
 1. RESIZED TO 96*96 PIXELS
 2. IMAGE IS CONVERTED TO AN ARRAY 
 3. The inputs pixel values are scaled between -1 and 1
 4. AND THEN APPENDED TO LIST 

In [6]:
data=[]
labels=[]

for i in imagePaths:
    label=i.split(os.path.sep)[-2]
    labels.append(label)
    image = load_img(i,target_size=(96,96))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

C:\Users\91890\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\PIL\Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


LABELS AND DATA IS CONVERTED TO AN NUMPY ARRAY 

In [7]:
data = np.array(data,dtype='float32')
labels = np.array(labels)

In [8]:
labels

array(['without_mask', 'without_mask', 'without_mask', ..., 'with_mask',
       'with_mask', 'with_mask'], dtype='<U12')

THE LABELED DATA 'LABELS' WHICH IS IN STRING IS CONVERTED TO NUMPY ARRAY 

In [9]:
lb=LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [11]:
type(labels[1])

numpy.ndarray

In [12]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

DATASET IS SPLIT BETWEEN TRAINING AND TESTING DATASET 

In [13]:
## Splitting of Data

train_X,test_X,train_Y,test_Y = train_test_split(data,labels,test_size=0.20,random_state=10,stratify=labels)

TRAINING IMAGE DATA 

In [14]:
train_X

array([[[[-0.90588236, -0.90588236, -0.92156863],
         [-0.90588236, -0.90588236, -0.92156863],
         [-0.9137255 , -0.8980392 , -0.92156863],
         ...,
         [-0.26274508, -0.60784316, -0.6784314 ],
         [-0.25490195, -0.5921569 , -0.6627451 ],
         [-0.26274508, -0.6       , -0.67058825]],

        [[-0.90588236, -0.8980392 , -0.9372549 ],
         [-0.9137255 , -0.8980392 , -0.9372549 ],
         [-0.9137255 , -0.8980392 , -0.9372549 ],
         ...,
         [-0.27058822, -0.6156863 , -0.6862745 ],
         [-0.26274508, -0.6       , -0.67058825],
         [-0.26274508, -0.6       , -0.67058825]],

        [[-0.90588236, -0.8980392 , -0.9372549 ],
         [-0.9137255 , -0.8980392 , -0.9372549 ],
         [-0.9137255 , -0.8980392 , -0.9372549 ],
         ...,
         [-0.26274508, -0.60784316, -0.6784314 ],
         [-0.26274508, -0.6       , -0.67058825],
         [-0.27058822, -0.60784316, -0.6784314 ]],

        ...,

        [[-0.4823529 , -0.70980394, -0

TRAINING LABELS OF THE RESPECTIVE IMAGES

In [15]:
train_Y

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

OUT OF 4000 IMAGES EACH OF 2KB THE SIZE OF TRAINING DATA IS 80% = 6400  

In [16]:
train_Y.size

6400

OUT OF 4000 IMAGES EACH OF 2KB THE SIZE OF TESTING DATA IS 20% = 1600  

In [17]:
test_Y.size

1600

DATA IS AUGUMENTED TO GENRATE VARIOUS VERSIONS OF THE SAME PICTURES 

In [18]:
aug=ImageDataGenerator(
                       rotation_range=20,
                       zoom_range=0.15,width_shift_range=0.2,
                       height_shift_range=0.2,shear_range=0.15,
                       horizontal_flip=True,
                       vertical_flip=True,
                       fill_mode='nearest'
                       )

INPUT SHAPE OF IMAGE = 96*96 PIXELS ; *3 FOR RGB 

In [20]:
input_shape = (96,96,3)
type(input_shape)
input_shape

(96, 96, 3)

LAYERS - 14 
5 CONV 2D 
5 MAXPOOLING 
1 FLATTEN 
3 DENSE 

In [21]:
# Build Model
input_image = Input(shape=input_shape)
# 1st Conv layer
model = Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape)(input_image)
model = MaxPooling2D((2, 2),padding='same')(model)
# 2nd Conv layer
model = Conv2D(32, (3, 3), activation='relu', padding='same')(model)
model = MaxPooling2D((2, 2),padding='same')(model)
# 3rd Conv layer
model = Conv2D(64, (3, 3), activation='relu', padding='same')(model)
model = MaxPooling2D((2, 2),padding='same')(model)
# 4th Conv layer
model = Conv2D(128, (3, 3), activation='relu', padding='same')(model)
model = MaxPooling2D((2, 2),padding='same')(model)
# 5th Conv layer
model = Conv2D(256, (3, 3), activation='relu', padding='same')(model)
model = MaxPooling2D((2, 2),padding='same')(model)
# FC layers
model = Flatten()(model)
model = Dense(1024)(model)
model = Dense(64)(model)
output= Dense(2, activation='softmax')(model)
model = Model(inputs=[input_image], outputs=[output])

In [22]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        18496 

BS = BATCH SIZE = THE NUMBER OF IMAGES THE MODEL TRAINS THROUGH BEFORE UPDATING ITS HYPERPARAMETERS
EPOCHS = THE NUMBER OF TIMES THE MODEL COMPLETELY RUNS THROUGH THE ENTIRE DATASET
ADAM OPTIMIZER IS USED 

In [24]:
lr=0.0005
Epochs=100
BS=32

opt=Adam(learning_rate=lr,decay=lr/Epochs)

BINARY CROSSENTROPY = Measures the probability error in discrete classification

In [25]:
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

EPOCHS TRAINED 

In [26]:
history=model.fit(
aug.flow(train_X,train_Y,batch_size=BS),
steps_per_epoch=len(train_X)//BS,
validation_data=(test_X,test_Y),
validation_steps=len(test_X)//BS,
epochs=Epochs)


Epoch 1/5
100/100 [==============================] - 18s 174ms/step - loss: 0.5520 - accuracy: 0.7209 - val_loss: 0.3690 - val_accuracy: 0.8338
Epoch 2/5
100/100 [==============================] - 17s 172ms/step - loss: 0.3784 - accuracy: 0.8319 - val_loss: 0.3069 - val_accuracy: 0.8838
Epoch 3/5
100/100 [==============================] - 17s 168ms/step - loss: 0.3178 - accuracy: 0.8697 - val_loss: 0.2120 - val_accuracy: 0.9212
Epoch 4/5
100/100 [==============================] - 17s 169ms/step - loss: 0.2680 - accuracy: 0.8916 - val_loss: 0.2104 - val_accuracy: 0.9150
Epoch 5/5
100/100 [==============================] - 17s 169ms/step - loss: 0.2514 - accuracy: 0.9016 - val_loss: 0.1970 - val_accuracy: 0.9175


SAVING THE MODEL 

In [27]:
model.save('model/custom_4000_32_222.h5')

In [28]:
predict=model.predict(test_X,batch_size=BS)
predict=np.argmax(predict,axis=1)
print(classification_report(test_Y.argmax(axis=1),predict,target_names=lb.classes_))

25/25 [==============================] - 1s 31ms/step
              precision    recall  f1-score   support

   with_mask       0.89      0.95      0.92       400
without_mask       0.95      0.88      0.91       400

    accuracy                           0.92       800
   macro avg       0.92      0.92      0.92       800
weighted avg       0.92      0.92      0.92       800

